# 03 – Transformación: channels_snapshot

Este notebook transforma el JSON crudo del recurso `channels`
en una tabla de snapshot mensual (`channels_snapshot`) lista
para análisis histórico y carga en BigQuery.

In [ ]:
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent
PROJECT_ROOT

In [ ]:
import json
import pandas as pd
from datetime import datetime, date

In [ ]:
RAW_PATH = PROJECT_ROOT / "data" / "raw" / "youtube"
PROCESSED_PATH = PROJECT_ROOT / "data" / "processed" / "youtube"

RAW_PATH, PROCESSED_PATH


In [ ]:
PROCESSED_PATH.mkdir(parents=True, exist_ok=True)


In [ ]:
with open(RAW_PATH / "channels.json", "r", encoding="utf-8") as f:
    channels_raw = json.load(f)

channels_raw.keys()


In [ ]:
item = channels_raw["items"][0]
item.keys()


In [ ]:
from datetime import date, datetime, timezone

# Fecha de snapshot (solo fecha, sin hora)
SNAPSHOT_DATE = date.today()

# Timestamp explícitamente en UTC (timezone-aware)
EXTRACTED_AT = datetime.now(timezone.utc)

SNAPSHOT_DATE, EXTRACTED_AT

In [ ]:
row_snapshot = {
    "snapshot_date": SNAPSHOT_DATE,
    "channel_id": item["id"],
    "subscriber_count": int(item["statistics"].get("subscriberCount", 0)),
    "view_count": int(item["statistics"].get("viewCount", 0)),
    "video_count": int(item["statistics"].get("videoCount", 0)),
    "extracted_at": EXTRACTED_AT
}

df_channels_snapshot = pd.DataFrame([row_snapshot])

df_channels_snapshot


In [ ]:
df_channels_snapshot.shape

In [ ]:
df_channels_snapshot.dtypes

Nota: snapshot_date  -> Los objetos datetime.date Se almacenan como dtype object, Esto es normal. No es un string. No es un error. Es simplemente cómo pandas maneja date.

In [ ]:
type(df_channels_snapshot["snapshot_date"].iloc[0])

In [ ]:
df_channels_snapshot = df_channels_snapshot[
    [
        "snapshot_date",
        "channel_id",
        "subscriber_count",
        "view_count",
        "video_count",
        "extracted_at"
    ]
]

> ⚠️ Nota (fase de desarrollo)
>
> El guardado en formato Parquet se utiliza temporalmente para pruebas y separación entre notebooks.
> En la versión final del pipeline (.py), este paso será omitido y el DataFrame se enviará directamente a BigQuery, donde residirá el histórico definitivo.

In [ ]:
output_file = PROCESSED_PATH / "channels_snapshot.parquet"

df_channels_snapshot.to_parquet(output_file, index=False)

output_file


In [ ]:
import pandas as pd
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent
file_path = PROJECT_ROOT / "data" / "processed" / "youtube" / "channels_snapshot.parquet"

df = pd.read_parquet(file_path)
df
